In [72]:
import numpy as np
from collections import namedtuple
from scipy.optimize import minimize_scalar
from scipy.linalg import solve_triangular
import numdifftools as ndt
from datetime import datetime as dt
from collections import namedtuple

In [70]:
# paraboloid, booth, rosenbrock, beales, matyas, ackleys
Pair = namedtuple('FPair', ['f', 'min_val', 'dim'])
functions = {
    'paraboloid': Pair(lambda v: v[0] ** 2 + v[1] ** 2, 0, 2),
    'booth': Pair(lambda v: np.sum(np.array([(v[0] + 2 * v[1] - 7) ** 2, (2 * v[0] + v[1] - 5) ** 2])), 0, 2),
    'rosenbrock': Pair(lambda v: np.sum([(1 - v[1]) ** 2, 100 * (v[0] - v[1] * v[1]) ** 2]), 0, 2),
    'beales': Pair(lambda v: np.sum([(1.5 - v[0] + v[0] * v[1]) ** 2, (2.25 - v[0] + v[0] * v[1] ** 2) ** 2, (2.625 - v[0] + v[0] * v[1] ** 3) ** 2]), 0, 2),
    'matyas': Pair(lambda v: np.sum([.26 * (v[0] ** 2 + v[1] ** 2),  - .48 * v[0] * v[1]]), 0, 2),
    'ackleys': Pair(lambda v: np.sum([-20 * np.exp(-.2 * np.sqrt(.5 * v[0] ** 2 + v[1] ** 2)), - np.exp(.5 * (np.cos(2 * np.pi * v[0]) + np.cos(2 * np.pi * v[0]))), np.e, 20]), 0, 2),
    'easom': Pair(lambda v: (-np.cos(v[0]) * np.cos(v[1]) * np.exp(-((v[0] - np.pi) ** 2 + (v[1] - np.pi) ** 2))), -1, 2),
    ''
}

In [71]:
default_params = ["MaxNumIter", "tolGrad", "tolIter", "gradHess", "alpha", "beta", "theta"]
p = dict(zip(default_params, len(default_params) * [None]))
p["MaxNumIter"] = 1000
p["tolGrad"] = 10e-5
p["tolIter"] = 10e-6
p["alpha"] = 10e-5

In [73]:
class MinAlgorithm:
    
    def __init__(self, f, grad=None, hess=None):
        self.func = f
        self.grad = grad if grad else ndt.Gradient(f)
        self.hess = hess if hess else ndt.Hessian(f)
        
    def minimize(self, starting_point, params):
        raise NotImplementedError()
        
    @classmethod
    def run_test_suites(cls, init_func=lambda x: x, params=p, runs=10):
        result_dict = {}
        for name, pair in functions.items():
            minimizer = cls(pair.f)
            init_func(minimizer)
            steps = []
            run_times = []
            approx = []
            for i in range(runs):
                start_point = np.random.randint(1, 100) * (np.random.rand(2, 1) - .5)
                before = dt.now()
                minimizer.minimize(start_point.reshape((-1, 1)), params)
                after = dt.now()
                steps.append(minimizer.steps)
                run_times.append((after - before).total_seconds())
                approx.append((minimizer.func(minimizer.solution) - pair.min_val) ** 2)
            result_dict[name] = {
                'time': (np.min(run_times), np.max(run_times), np.average(run_times)),
                'avg_steps': (np.min(steps), np.max(steps), np.average(steps)),
                'squared_error': (np.min(approx), np.max(approx), np.average(approx))
            }
        print(result_dict)
        return result_dict

In [6]:
OptResult = namedtuple('Result', ['x']) 
def ternary_search(f, interval, eps=10e-3):
    max_iter = 10
    steps = 0
    left, right = interval
    while steps < max_iter and np.abs(right - left) > eps:
        steps += 1
        mid_left = left + (right - left) / 3
        mid_right = right - (right - left) / 3
        if f(mid_left) < f(mid_right):
            right = mid_right
        else:
            left = mid_left
    return OptResult((right - left) / 2)

f = lambda x: x ** 2
ternary_search(f, (-3, 3))

Result(x=0.052024589747497844)

In [7]:
class SteepestDescent(MinAlgorithm):
    
    def set_line_search(self, name):
        if name == "fminsearch":
            self.line_search = lambda f, interval, eps: minimize_scalar(f, method="brent")
        elif name == "paso_limitado":
            self.line_search = None
        elif name == "orden_cero":
            self.line_search = lambda f, interval, eps: ternary_search(f, interval, eps)
        elif name == "armijo":
            self.line_search = 'armijo'
        else:
            raise Exception("El metodo de busqueda {} no existe".format(name))
        self.name = name
            
    def line_minimize(self, g, interval, eps=10e-3):
        res = self.line_search(g, interval, eps)
        return res.x
        
    def minimize(self, starting_point, params):
        assert getattr(self, "line_search", None)
        point, old_point = starting_point, starting_point + 1
        steps = 0
        grad = self.grad(point).reshape((-1, 1))
        d = -grad
        result = 100
        while steps < params["MaxNumIter"] and result > params["tolGrad"]\
                    and np.linalg.norm(point - old_point) > params["tolIter"]:
            steps += 1
            phi = lambda alpha: self.func((point + alpha * d))
            if self.name != 'armijo':
                min_alpha = self.line_minimize(phi, (-3, 3), params["alpha"])
            else:
                epsilon = params['alpha']
                eta = params['beta']
                min_alpha = np.random.rand()
                while True:
                    top = self.func(point) + epsilon * grad.T.dot(d) * min_alpha
                    bottom = self.func(point) + epsilon * grad.T.dot(d) * eta * min_alpha
                    if phi(min_alpha) > top:
                        min_alpha /= eta
                    elif phi(eta * min_alpha) <= bottom:
                        min_alpha *= eta
                    else:
                        break
            old_point = point
            point = point + min_alpha * d
            grad = self.grad(point).reshape((-1, 1))
            d = -grad
            result = np.linalg.norm(d)
        self.steps = steps
        self.solution = point
        return point

In [8]:
class QuadraticCongujateGradients(MinAlgorithm):
    
    def __init__(self, Q, b):
        self.Q = Q
        self.b = b
        self.g = lambda x: self.Q.dot(np.matrix(x)) + self.b
    
    def quadratic_minimize(self, starting_point, params):
        point, old_point = starting_point, starting_point + 1
        steps = 0
        d = - self.g(point)
        result = 100
        while steps < params["MaxNumIter"] and result > params["tolGrad"]\
                and np.linalg.norm(point - old_point) > params["tolIter"]:
            steps += 1
            alpha = - float((self.g(point).T.dot(d)) / d.T.dot(self.Q.dot(d)))
            old_point = point
            point = point + alpha * d
            new_grad = self.g(point)
            beta = float((new_grad.T.dot(self.Q.dot(d))) / d.T.dot(self.Q.dot(d)))
            d = -new_grad + beta * d
            result = np.linalg.norm(new_grad)
            print(result)
        self.steps = steps
        self.solution = point
        return point

In [66]:
dim = 50
L = np.tril(np.random.rand(dim, dim) + 10)
Q = L.dot(L.T)
b = np.random.rand(dim, 1)
quad_min = QuadraticCongujateGradients(Q, b)
result = quad_min.quadratic_minimize(np.random.rand(dim, 1) + 20, p)

512505.766256
84339.2177369
24526.7027657
9225.61062231
4048.7503179
2017.86079286
1121.26577654
950.701018401
752.36763984
354.252445891
202.13045821
121.529895917
156.520273158
80.0701915568
49.5430558412
151.017719384
25.8904704547
25.4830462273
23.5151148771
8.61539332228
4.79557301045
77.0569035593
3.25663009678
5.48222417283
4.35574437097
1.74181847258
0.859217950749
0.97535864348
1.57018171858
0.515004180347
0.245366689077
0.838414652737
0.237923332486
0.142424517178
2.32201334627
0.114084391894
0.068361312924
0.0712551793518
0.021709459441
0.0649066692232
0.0143324731511
0.00728502855956


In [9]:
class RestartCongujateGradients(MinAlgorithm):
    
    def minimize(self, starting_point, params):
        point, old_point = starting_point, starting_point + 1
        steps = 0
        grad = self.grad(point).reshape((-1, 1))
        d = - grad
        result = 100
        n = len(starting_point)
        while steps < params["MaxNumIter"] and result > params["tolGrad"]\
                and np.linalg.norm(point - old_point) > params["tolIter"]:
            steps += 1
            res = minimize_scalar(lambda alpha: self.func(point + alpha * d), method="brent")
            alpha = res.x
            old_point = point
            point = point + alpha * d
            old_grad = grad
            grad = self.grad(point).reshape((-1, 1))
            if steps % n > 0:
                beta = float(((grad - old_grad).T.dot(grad)) / old_grad.T.dot(old_grad))
                d = -grad + beta * d
            else:
                d = -grad
            result = np.linalg.norm(d)
        self.steps = steps
        self.solution = point
        return point

In [74]:
class GlobalConvergenceNewtonMethod(MinAlgorithm):
    
    def minimize(self, starting_point, params):
        point, old_point = starting_point, starting_point + 1
        steps = 0
        result = 100
        Id = np.eye(len(point))
        modified_hess = lambda mu: np.matrix((mu * Id) + hess)
        while steps < params["MaxNumIter"] and result > params["tolGrad"]\
                and np.linalg.norm(point - old_point) > params["tolIter"]:
            steps += 1
            grad = self.grad(point).reshape((-1, 1))
            hess = self.hess(point.flatten())
            mu = 0
            arranque = True
            while arranque:
                new_hess = modified_hess(mu)
                try:
                    L = np.linalg.cholesky(new_hess)
                    y = solve_triangular(L, grad)
                    d = -solve_triangular(L.H, y)
                    arranque = False
                except np.linalg.LinAlgError:
                    mu = 10 * mu if mu > 0 else 1
            phi = lambda alpha: self.func((point + alpha * d))
            res = minimize_scalar(phi, method='brent')
            min_alpha = res.x
            old_point = point
            point = point + min_alpha * d
            result = np.linalg.norm(grad)
        self.steps = steps
        self.solution = point
        return point

In [75]:
GlobalConvergenceNewtonMethod.run_test_suites(runs=10)

{'paraboloid': {'time': (0.014437, 0.020778999999999999, 0.017735800000000003), 'avg_steps': (2, 2, 2.0), 'squared_error': (0.0, 2.6625244848471646e-110, 2.8296729625226754e-111)}, 'easom': {'time': (0.0092659999999999999, 0.035955000000000001, 0.013925199999999999), 'avg_steps': (1, 4, 1.3999999999999999), 'squared_error': (0.0, 1.0, 0.89999999999890457)}, 'ackleys': {'time': (0.051555999999999998, 0.077322000000000002, 0.066997000000000001), 'avg_steps': (4, 5, 4.2999999999999998), 'squared_error': (2.5106728931670845e-22, 395.09441584736487, 141.85680564124343)}, 'rosenbrock': {'time': (0.40304600000000002, 0.95855599999999996, 0.61260710000000007), 'avg_steps': (36, 86, 52.899999999999999), 'squared_error': (0.045026148128548819, 0.35602469064200465, 0.22315562764287442)}, 'matyas': {'time': (0.127051, 0.150195, 0.14105519999999999), 'avg_steps': (12, 13, 12.699999999999999), 'squared_error': (1.0844544033867869, 222753.88108723052, 56276.37936514341)}, 'beales': {'time': (0.105645

{'ackleys': {'avg_steps': (4, 5, 4.2999999999999998),
  'squared_error': (2.5106728931670845e-22,
   395.09441584736487,
   141.85680564124343),
  'time': (0.051555999999999998, 0.077322000000000002, 0.066997000000000001)},
 'beales': {'avg_steps': (9, 61, 37.399999999999999),
  'squared_error': (0.00012337479597527583,
   1.5524357269794402,
   0.29004277189986022),
  'time': (0.105645, 0.74533000000000005, 0.44416060000000002)},
 'booth': {'avg_steps': (11, 20, 16.100000000000001),
  'squared_error': (4.4914623672126818e-27,
   6.9578218746958592e-19,
   1.8383920215783191e-19),
  'time': (0.11811199999999999, 0.213008, 0.16912099999999997)},
 'easom': {'avg_steps': (1, 4, 1.3999999999999999),
  'squared_error': (0.0, 1.0, 0.89999999999890457),
  'time': (0.0092659999999999999, 0.035955000000000001, 0.013925199999999999)},
 'matyas': {'avg_steps': (12, 13, 12.699999999999999),
  'squared_error': (1.0844544033867869, 222753.88108723052, 56276.37936514341),
  'time': (0.127051, 0.15019

In [10]:
RestartCongujateGradients.run_test_suites(runs=20)

/usr/local/lib/python3.5/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:19: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:19

{'paraboloid': {'time': (0.006071, 0.031607000000000003, 0.0089343499999999989), 'avg_steps': (1, 1, 1.0), 'squared_error': (2.4308653429145085e-63, 9.7469570282173448e-48, 4.8751730823171833e-49)}, 'easom': {'time': (0.007744, 0.033447999999999999, 0.01208685), 'avg_steps': (1, 5, 1.55), 'squared_error': (0.0, 1.0, 0.85044356436199231)}, 'ackleys': {'time': (0.025739999999999999, 0.092252000000000001, 0.051411750000000013), 'avg_steps': (3, 11, 6.0), 'squared_error': (7.5058789340587523e-21, 399.32138709359549, 139.96986093879201)}, 'rosenbrock': {'time': (0.031526999999999999, 0.356487, 0.12826535), 'avg_steps': (5, 58, 22.649999999999999), 'squared_error': (4.5937188959752823e-30, 4.5658374821303532e-13, 2.2837959235398611e-14)}, 'matyas': {'time': (0.012012999999999999, 0.013232000000000001, 0.012491800000000001), 'avg_steps': (2, 2, 2.0), 'squared_error': (3.3382623248902707e-58, 1.4102633425917444e-27, 8.4887300943506024e-29)}, 'beales': {'time': (0.017361000000000001, 0.52829300

{'ackleys': {'avg_steps': (3, 11, 6.0),
  'squared_error': (7.5058789340587523e-21,
   399.32138709359549,
   139.96986093879201),
  'time': (0.025739999999999999, 0.092252000000000001, 0.051411750000000013)},
 'beales': {'avg_steps': (2, 83, 19.399999999999999),
  'squared_error': (2.7889346112279576e-24,
   0.30937726193092291,
   0.099271950416072507),
  'time': (0.017361000000000001, 0.52829300000000001, 0.1218011)},
 'booth': {'avg_steps': (2, 2, 2.0),
  'squared_error': (1.5557538194652854e-59,
   9.3877368698082234e-27,
   4.7073957804806746e-28),
  'time': (0.011682, 0.021583000000000001, 0.0138718)},
 'easom': {'avg_steps': (1, 5, 1.55),
  'squared_error': (0.0, 1.0, 0.85044356436199231),
  'time': (0.007744, 0.033447999999999999, 0.01208685)},
 'matyas': {'avg_steps': (2, 2, 2.0),
  'squared_error': (3.3382623248902707e-58,
   1.4102633425917444e-27,
   8.4887300943506024e-29),
  'time': (0.012012999999999999, 0.013232000000000001, 0.012491800000000001)},
 'paraboloid': {'avg

In [63]:
p['alpha'] = .2
p['beta'] = 2
SteepestDescent.run_test_suites(lambda x: x.set_line_search('armijo'), runs=10, params=p)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in cos
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:29: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in cos


{'paraboloid': {'squared_error': (8.2852777821528722e-20, 6.1452651018690685e-18, 2.1564928829580641e-18), 'avg_steps': (4, 9, 6.5999999999999996), 'time': (0.01831, 0.048481999999999997, 0.030047899999999999)}, 'easom': {'squared_error': (1.0, 1.0, 1.0), 'avg_steps': (1, 1, 1.0), 'time': (0.043388999999999997, 0.11862, 0.098422699999999988)}, 'booth': {'squared_error': (2.4673313934622814e-21, 3.2341836977591995e-18, 4.4263097006020543e-19), 'avg_steps': (21, 41, 29.699999999999999), 'time': (0.101497, 0.20441500000000001, 0.15076829999999999)}, 'ackleys': {'squared_error': (6.3017620076565991e-06, 399.99996374080604, 309.26114533273596), 'avg_steps': (9, 1000, 597.29999999999995), 'time': (0.066070000000000004, 6.4966619999999997, 3.7960766000000006)}, 'matyas': {'squared_error': (2.7246684083449153e-19, 1.0155455901487475e-14, 3.1958720800158265e-15), 'avg_steps': (24, 62, 42.799999999999997), 'time': (0.14014399999999999, 0.331038, 0.23823729999999999)}, 'rosenbrock': {'squared_err

{'ackleys': {'avg_steps': (9, 1000, 597.29999999999995),
  'squared_error': (6.3017620076565991e-06,
   399.99996374080604,
   309.26114533273596),
  'time': (0.066070000000000004, 6.4966619999999997, 3.7960766000000006)},
 'beales': {'avg_steps': (7, 1000, 163.19999999999999),
  'squared_error': (8.976309805612612e-17,
   46.886469884859117,
   9.1813639578883262),
  'time': (0.067169999999999994, 6.0364699999999996, 0.98431920000000006)},
 'booth': {'avg_steps': (21, 41, 29.699999999999999),
  'squared_error': (2.4673313934622814e-21,
   3.2341836977591995e-18,
   4.4263097006020543e-19),
  'time': (0.101497, 0.20441500000000001, 0.15076829999999999)},
 'easom': {'avg_steps': (1, 1, 1.0),
  'squared_error': (1.0, 1.0, 1.0),
  'time': (0.043388999999999997, 0.11862, 0.098422699999999988)},
 'matyas': {'avg_steps': (24, 62, 42.799999999999997),
  'squared_error': (2.7246684083449153e-19,
   1.0155455901487475e-14,
   3.1958720800158265e-15),
  'time': (0.14014399999999999, 0.331038, 0.